# 教師あり二値分類（クラス9のみ正）のノートブック

In [1]:
from matplotlib import pyplot as plt
import polars as pl
import seaborn as sns
import japanize_matplotlib

from common import *

In [2]:
# ハイパーパラメータ
N_COMPONENTS = 50
TEST_SIZE = 0.2
VAL_SIZE = 0.2

In [3]:
# データセットの読み込み
X_train, X_val, X_test, y_train, y_val, y_test = load_mnist_pca_train_test_val(
    n_components=N_COMPONENTS, test_size=TEST_SIZE, val_size=VAL_SIZE
)

Files already downloaded
Files already downloaded


In [4]:
# ラベルの変更（クラス9のみ正）
y_train = (y_train == 9).astype(int)
y_val = (y_val == 9).astype(int)
y_test = (y_test == 9).astype(int)

In [5]:
def train_lgbm_with_custom_loss(X_train, X_valid, y_train, y_valid):

    """Train LightGBM with custom loss function."""
    train_data = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
    valid_data = lgb.Dataset(X_valid, label=y_valid, free_raw_data=False)

    params = {
        "objective": "binary",
        "verbose": -1,
        "learning_rate": 0.1,
        "num_boost_round": 200,
    }
    valid_accuracies = []

    def record_accuracies(p: lgb.Booster, train_data: lgb.Dataset, valid_data: lgb.Dataset):
        valid_pred = (p.predict(valid_data.data) > 0.5).astype(int)
        valid_acc = accuracy_score(valid_data.label, valid_pred)
        valid_accuracies.append(valid_acc)
        print(valid_acc)
    
    gbm = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        callbacks=[lambda p: record_accuracies(p.model, train_data, valid_data)]
    )
    return gbm, valid_accuracies

In [6]:
gbm, valid_accuracies =train_lgbm_with_custom_loss(X_train, X_val, y_train, y_val)
y_pred = gbm.predict(X_val)
print("Accuracy: {}".format(accuracy_score(y_val, (y_pred > 0.5).astype(int))))

/Users/ryoma.kobayashi/gits/techblog_git/wsl_w_lgbm/.venv/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


0.9043333333333333
0.9043333333333333
0.9043333333333333
0.9043333333333333
0.9055
0.92975
0.93825
0.9440833333333334
0.94925
0.9528333333333333
0.95525
0.9581666666666667
0.9595833333333333
0.9604166666666667
0.9615
0.9638333333333333
0.9648333333333333
0.96575
0.9669166666666666
0.9674166666666667
0.9681666666666666
0.9694166666666667
0.9698333333333333
0.9705
0.9713333333333334
0.97125
0.9723333333333334
0.97275
0.973
0.9735
0.974
0.9740833333333333
0.9746666666666667
0.9746666666666667
0.97525
0.976
0.9764166666666667
0.97675
0.9766666666666667
0.97725
0.97725
0.9770833333333333
0.9776666666666667
0.97825
0.9783333333333334
0.9781666666666666
0.9780833333333333
0.9785
0.979
0.9790833333333333
0.979
0.9795833333333334
0.9799166666666667
0.9800833333333333
0.9805833333333334
0.9805833333333334
0.9805
0.98075
0.9811666666666666
0.9808333333333333
0.9810833333333333
0.9810833333333333
0.9815
0.9814166666666667
0.9818333333333333
0.98175
0.9819166666666667
0.98175
0.98175
0.981916666666

In [7]:
np.save("sup_bin9_valid_accuracies.npy", valid_accuracies)